In [3]:
import os
import sys
import autogen
import pandas as pd
from typing import Dict
import pulp
import json


# Define the LLM configuration
llm_config = {
    "model": "gpt-4-turbo",
    "api_key": os.environ.get("OPENAI_API_KEY"),
    "temperature": 0,
    "seed": 42,
}

# Custom function to handle agent output
def agent_output(agent, message):
    if agent.name == "UserInputAgent":
        try:
            constraints = json.loads(message)
            if constraints:
                print(f"Processed constraints: {constraints}")
            else:
                print("No specific constraints. Using default settings.")
        except json.JSONDecodeError:
            print("Error processing input. Using default settings.")
    elif agent.name == "LineupOptimizer":
        print(message)

# User Input Agent
user_input_agent = autogen.AssistantAgent(
    name="UserInputAgent",
    llm_config=llm_config,
    system_message="""You are an agent that processes user requests for fantasy football lineups.
    Parse the user's input and convert it into a structured format for lineup optimization.
    If the input is empty or doesn't contain specific constraints, return an empty dictionary.
    Use the following examples as a guide:

    Input: "I want Justin Jefferson in my lineup"
    Output: {"must_include": ["justinjefferson"]}

    Input: "Can you make sure to prioritize running backs?"
    Output: {"position_emphasis": {"RB": 2.0}}

    Input: "I'd like to focus on players from the Vikings"
    Output: {"team_preference": "MIN"}

    Input: "Generate me 2 unique lineups"
    Output: {"num_lineups": 2}

    Input: "Create 3 lineups with different strategies"
    Output: {"num_lineups": 3, "diverse_strategies": true}

    Input: "" (empty input)
    Output: {}

    Provide your output as a valid JSON object."""
)

# Lineup Optimizer Agent
class LineupOptimizerAgent(autogen.AssistantAgent):
    def __init__(self, name, data_file, llm_config):
        super().__init__(name=name, llm_config=llm_config)
        self.data = pd.read_csv(data_file)
        self.data['salary'] = self.data['salary'].replace('[\$,]', '', regex=True).astype(float)
        self.data['projected_points'] = pd.to_numeric(self.data['projected_points'], errors='coerce')
        self.previous_lineups = []

    def optimize_lineup(self, constraints: Dict) -> Dict:
        players = self.data.to_dict('records')
        prob = pulp.LpProblem("Fantasy Football", pulp.LpMaximize)
        
        player_vars = pulp.LpVariable.dicts("players", 
                                            ((p['player_name'], p['player_position_id']) for p in players), 
                                            cat='Binary')

        # Objective: Maximize total projected points
        prob += pulp.lpSum([player['projected_points'] * player_vars[player['player_name'], player['player_position_id']] 
                            for player in players])

        # Constraints
        prob += pulp.lpSum([player['salary'] * player_vars[player['player_name'], player['player_position_id']] for player in players]) <= 50000
        prob += pulp.lpSum([player_vars[p['player_name'], 'QB'] for p in players if p['player_position_id'] == 'QB']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], 'RB'] for p in players if p['player_position_id'] == 'RB']) >= 2
        prob += pulp.lpSum([player_vars[p['player_name'], 'WR'] for p in players if p['player_position_id'] == 'WR']) >= 3
        prob += pulp.lpSum([player_vars[p['player_name'], 'TE'] for p in players if p['player_position_id'] == 'TE']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], 'DST'] for p in players if p['player_position_id'] == 'DST']) == 1
        prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] for p in players]) == 9

        # Apply constraints from user input
        if 'must_include' in constraints:
            for player_name in constraints['must_include']:
                prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                    for p in players if p['player_name'].lower() == player_name.lower()]) == 1

        if 'position_emphasis' in constraints:
            for position, emphasis in constraints['position_emphasis'].items():
                prob += pulp.lpSum([player['projected_points'] * player_vars[p['player_name'], position] 
                                    for p in players if p['player_position_id'] == position]) >= emphasis * prob.objective

        if 'team_preference' in constraints:
            team = constraints['team_preference']
            prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                for p in players if p['player_team_id'] == team]) >= 3

        # Exclude players from previous lineups
        for prev_lineup in self.previous_lineups:
            prob += pulp.lpSum([player_vars[p['player_name'], p['player_position_id']] 
                                for p in prev_lineup.values() if p is not None]) <= 6  # Allow up to 6 players to overlap

        prob.solve()

        lineup = {
            'QB': None, 'RB1': None, 'RB2': None, 'WR1': None, 'WR2': None, 'WR3': None, 
            'TE': None, 'FLEX': None, 'DST': None
        }

        for player in players:
            if player_vars[player['player_name'], player['player_position_id']].value() == 1:
                position = player['player_position_id']
                if position == 'QB' and lineup['QB'] is None:
                    lineup['QB'] = player
                elif position == 'RB':
                    if lineup['RB1'] is None:
                        lineup['RB1'] = player
                    elif lineup['RB2'] is None:
                        lineup['RB2'] = player
                    else:
                        lineup['FLEX'] = player
                elif position == 'WR':
                    if lineup['WR1'] is None:
                        lineup['WR1'] = player
                    elif lineup['WR2'] is None:
                        lineup['WR2'] = player
                    elif lineup['WR3'] is None:
                        lineup['WR3'] = player
                    else:
                        lineup['FLEX'] = player
                elif position == 'TE':
                    if lineup['TE'] is None:
                        lineup['TE'] = player
                    else:
                        lineup['FLEX'] = player
                elif position == 'DST':
                    lineup['DST'] = player

        self.previous_lineups.append(lineup)
        return lineup

    def reset_lineups(self):
        self.previous_lineups = []

    def format_lineup(self, lineup):
        output = "Optimized Lineup:\n"
        total_salary = 0
        total_points = 0
        for position, player in lineup.items():
            if player:
                output += f"{position}: {player['player_name']} (Projected: {player['projected_points']:.2f}, Salary: ${player['salary']})\n"
                total_salary += player['salary']
                total_points += player['projected_points']
            else:
                output += f"{position}: Not filled\n"
        output += f"\nTotal Salary: ${total_salary}"
        output += f"\nTotal Projected Points: {total_points:.2f}"
        return output

optimizer_agent = LineupOptimizerAgent(
    name="LineupOptimizer",
    data_file='data/week18_sunday_all.csv',
    llm_config=llm_config
)

# User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=0
)

def fantasy_football_chat():
    print("Welcome to the Fantasy Football Lineup Generator!")
    print("You can enter specific requests or just press Enter to generate a lineup with default settings.")
    print("Type 'quit' to exit the program.")

    while True:
        try:
            user_input = input("\nEnter your lineup request (or press Enter for default, 'quit' to exit): ").strip()
            if user_input.lower() == 'quit':
                print("Exiting the program. Goodbye!")
                sys.exit(0)

            constraints = {}
            if user_input:
                print("\nUser Input Agent processing request...")
                user_proxy.send(
                    user_input,
                    user_input_agent,
                    request_reply=True
                )
                user_input_response = user_input_agent.last_message()['content']
                print(f"User Input Agent response: {user_input_response}")
                
                try:
                    constraints = json.loads(user_input_response)
                except json.JSONDecodeError:
                    print("Error processing input. Using default settings.")
            else:
                print("\nUsing default settings...")

            num_lineups = constraints.get('num_lineups', 1)
            optimizer_agent.reset_lineups()  # Reset previous lineups

            for i in range(num_lineups):
                print(f"\nGenerating lineup {i+1}...")
                lineup = optimizer_agent.optimize_lineup(constraints)
                formatted_lineup = optimizer_agent.format_lineup(lineup)
                print(f"\nGenerated Lineup {i+1}:")
                print(formatted_lineup)

            print("\n" + "-"*50)
        except KeyboardInterrupt:
            print("\nProgram interrupted. Exiting...")
            sys.exit(0)
        except Exception as e:
            print(f"An error occurred: {e}")

# Run the chat
if __name__ == "__main__":
    fantasy_football_chat()

Welcome to the Fantasy Football Lineup Generator!
You can enter specific requests or just press Enter to generate a lineup with default settings.
Type 'quit' to exit the program.

User Input Agent processing request...
User (to UserInputAgent):

Generate 2 lineups

--------------------------------------------------------------------------------
UserInputAgent (to User):

{"num_lineups": 2}

--------------------------------------------------------------------------------


/usr/local/lib/python3.11/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/usr/local/lib/python3.11/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


User Input Agent response: {"num_lineups": 2}

Generating lineup 1...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/44c9e272f25f41c4b101b181bc77b4c0-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/44c9e272f25f41c4b101b181bc77b4c0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 2137 RHS
At line 2145 BOUNDS
At line 2505 ENDATA
Problem MODEL has 7 rows, 359 columns and 1077 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 128.68 - 0.00 seconds
Cgl0004I processed model has 7 rows, 284 columns (284 integer (277 of which binary)) and 852 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.4
Cbc0038I Solution found of -127.4
Cbc0038I Cleaned solution of -127.4
C

SystemExit: 0

/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
